# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20902062


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/31 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/31 [00:04<00:59,  2.06s/it]

Rendering models:  10%|▉         | 3/31 [00:06<01:04,  2.29s/it]

Rendering models:  13%|█▎        | 4/31 [00:07<00:44,  1.65s/it]

Rendering models:  16%|█▌        | 5/31 [00:07<00:31,  1.19s/it]

...

Rendering models:  84%|████████▍ | 26/31 [00:14<00:01,  2.68it/s]

Rendering models:  87%|████████▋ | 27/31 [00:14<00:01,  2.96it/s]

Rendering models:  90%|█████████ | 28/31 [00:15<00:01,  2.94it/s]

Rendering models:  97%|█████████▋| 30/31 [00:15<00:00,  3.83it/s]

equidad1                              0.115082
not-logged-in-f492513263c5d12990d3    0.000471
tdickey0007                           0.002232
tartoso                               0.000999
Thedunkmasta                          0.003459
OregonHiker                           0.000310
mlang2017                             0.082494
graeme94                              0.000680
irwansyah                             0.004325
sinjinza1984                          0.016395
not-logged-in-ef256f13ca4c9a6bfcf7    0.001303
not-logged-in-c08c0c43bd0e8eef780a    0.001108
not-logged-in-f2eb212bec3c0ec6797b    0.033337
not-logged-in-3c5cb8813e4cf6584808    0.006518
not-logged-in-9a45911eb71184fe7c7f    0.000550
Mckay_Oyler                           0.000384
Saharisunshine                        0.004834
41Letters                             0.000327
not-logged-in-0bc167e074b89ab0e60b    0.000889
RaulE                                 0.005164
not-logged-in-20a090b00932d22636ca    0.000318
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())